In [109]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as tfs

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [110]:
import sys, os
cur_dir = os.getcwd()
test_dir = os.path.join(cur_dir, os.pardir)
src_dir = os.path.join(test_dir, os.pardir)
sys.path.append(os.path.abspath(src_dir))

In [111]:
from data.processed.CelebA_1024.call_celeba import CelebALoader
cl = CelebALoader()

In [157]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                    nn.GELU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                    nn.GELU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(152064, 4)

        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out     

    def predict_mbti(self, x):
        out = self.forward(x).squeeze(0)
        out = out - 0.5

        m = 'E' if out[0] > 0 else 'I'
        b = 'N' if out[1] > 0 else 'S'
        t = 'F' if out[2] > 0 else 'T'
        i = 'J' if out[3] > 0 else 'P'
        mbti = '{}{}{}{}'.format(m, b, t, i)
        return mbti


In [158]:
model = CNN()

In [159]:
los = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())

In [166]:
for epoch in range(5):
    opt.zero_grad()

    hypothesis = model(torch.randn(size=(1, 1, 218, 178)))
    y = torch.Tensor([[.5, .5, .5, .5]])
    cost = los(hypothesis, y)
    cost.backward()
    opt.step()

    print(cost.item())

20.15863037109375
12.447321891784668
12.754327774047852
14.052033424377441
19.232568740844727


In [167]:
img1 = cl.load_img(startIdx=1, endIdx=1, gray=True)[0].unsqueeze(0).unsqueeze(0)
img2 = cl.load_img(startIdx=2, endIdx=2, gray=True)[0].unsqueeze(0).unsqueeze(0)

In [168]:
model.forward(img2)

tensor([[ 1.0463, -2.3747,  1.6960,  1.3798]], grad_fn=<AddmmBackward0>)

In [169]:
model.predict_mbti(img1)

'ISFP'

In [170]:
model.predict_mbti(img2)

'ESFJ'

In [171]:
for n in range(10):
    img_n = cl.load_img(startIdx=n+1, endIdx=n+1, gray=True)[0].unsqueeze(0).unsqueeze(0)
    print(model.predict_mbti(img_n))

ISFP
ESFJ
ISFJ
ESFJ
ISFP
ISFP
ISFP
ESFJ


ESFJ
ESFJ
